# Elasticidad Precio de la Demanda


Partiendo de un volumen tan grande de datos y una estructura compleja como la que maneja DIARCO, la mejor estrategia sería una combinación de técnicas para abordar tanto la elasticidad precio de la demanda como la predicción de ventas futuras bajo cambios de precio. A continuación, se detalla un enfoque híbrido que incluye:

1. Análisis inicial y cálculo de elasticidades por segmentos
2. Entrenamiento de un modelo de machine learning para predicciones dinámicas
3. Integración de elasticidades calculadas y el modelo en un sistema escalable

## Paso 1: Preparación de Datos
### 1.1. Agregar Clasificaciones y Limpieza de Datos
Cargar los datos, asegurarse de que las claves (tienda, producto, fecha) sean únicas y agregar clasificaciones.

In [1]:
import pandas as pd

# Cargar datos
ventas = pd.read_csv("ventas.csv")  # Contiene columnas: tienda, producto, fecha, cantidad_vendida
precios = pd.read_csv("precios.csv")  # Contiene columnas: tienda, producto, fecha, precio_retail, promocion, folder
clasificacion = pd.read_csv("clasificacion_productos.csv")  # Contiene: producto, familia, rubro, sub_rubro, categoria_abc

# Merge de datos
data = pd.merge(ventas, precios, on=["tienda", "producto", "fecha"])
data = pd.merge(data, clasificacion, on="producto")

# Convertir fechas y ordenar
data["fecha"] = pd.to_datetime(data["fecha"])
data = data.sort_values(by=["producto", "tienda", "fecha"])


FileNotFoundError: [Errno 2] No such file or directory: 'ventas.csv'

### 1.2. Crear Segmentos para el Análisis
Agrupar productos por Familia, Rubro, Sub_Rubro y Categoría ABC. Esto permite calcular elasticidades específicas por segmento.

In [ ]:
# Crear etiquetas de clasificación
data["segmento"] = data["familia"] + "_" + data["rubro"] + "_" + data["categoria_abc"]


## Paso 2: Cálculo de Elasticidades por Segmento
Usar regresiones por cada segmento para estimar elasticidades.

### 2.1. Implementar Modelo de Regresión
Se utiliza una regresión en cada segmento para calcular elasticidades precio de la demanda.

In [ ]:
import numpy as np
import statsmodels.api as sm

# Crear una función para calcular elasticidades
def calcular_elasticidad(segmento, data):
    subset = data[data["segmento"] == segmento]
    subset["log_q"] = np.log(subset["cantidad_vendida"])
    subset["log_p"] = np.log(subset["precio_retail"])
    
    # Regresión lineal log-log
    X = sm.add_constant(subset["log_p"])
    y = subset["log_q"]
    modelo = sm.OLS(y, X).fit()
    
    return modelo.params["log_p"]

# Calcular elasticidades para todos los segmentos
segmentos = data["segmento"].unique()
elasticidades = {seg: calcular_elasticidad(seg, data) for seg in segmentos}

# Agregar elasticidades al dataset
data["elasticidad"] = data["segmento"].map(elasticidades)


## Paso 3: Entrenamiento de un Modelo Predictivo
### 3.1. Configurar Variables
Preparar las variables predictoras para entrenar un modelo que estime las ventas futuras.

In [ ]:
from sklearn.model_selection import train_test_split

# Selección de variables
data["promocion_bin"] = data["promocion"].astype(int)
X = data[["precio_retail", "promocion_bin", "folder", "elasticidad"]]
y = data["cantidad_vendida"]

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### 3.2. Entrenar un Modelo de Machine Learning
Entrenar un modelo de **XGBoost** para predecir las ventas bajo distintos escenarios.

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Modelo XGBoost
modelo_ml = XGBRegressor(objective="reg:squarederror", n_estimators=100, max_depth=6)
modelo_ml.fit(X_train, y_train)

# Evaluación
y_pred = modelo_ml.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE del modelo: {rmse}")


## Paso 4: Integrar Elasticidades y Modelo Predictivo
Combinar elasticidades calculadas y el modelo para simular el impacto de cambios de precio en las ventas.

### 4.1. Simulación de Escenarios
Por ejemplo, calcular cómo cambiarían las ventas si los precios aumentaran un 10%.

In [ ]:
# Simular incremento de precio
data["precio_simulado"] = data["precio_retail"] * 1.1
data["cantidad_simulada"] = data["cantidad_vendida"] * (1 + data["elasticidad"] * 0.1)

# Predicción usando el modelo
data["ventas_predichas"] = modelo_ml.predict(data[["precio_simulado", "promocion_bin", "folder", "elasticidad"]])

# Guardar resultados
data.to_csv("resultados_simulacion.csv", index=False)


## Paso 5: Escalabilidad
Para manejar millones de registros:

1. Dividir los datos por segmento y tienda.
    - Procesar segmentos o tiendas de forma independiente.
2. Uso de herramientas de procesamiento distribuido:
    - Usar frameworks como Dask o PySpark para paralelizar las operaciones.
3. Almacenamiento escalable:
    - Guardar elasticidades y resultados en una base de datos como PostgreSQL o BigQuery.


## Paso 6: Monitorización y Ajuste Continuo
Configurar un pipeline que actualice elasticidades y modelos regularmente:

1. **Reentrenar el modelo**: Cada mes con datos recientes.
2. **Actualizar elasticidades**: Recalcular para productos o segmentos con cambios significativos en patrones de ventas.

## Conclusión
Este enfoque híbrido permite:

- Cálculo de elasticidades: Para decisiones estratégicas.
- Modelos predictivos dinámicos: Para simulaciones de escenarios futuros.
- Escalabilidad: Procesar millones de registros de manera eficiente.